In [8]:
import os
import dotenv
from datasets import load_dataset, DatasetDict
from huggingface_hub import login, HfApi
dotenv.load_dotenv()

NEW_DATASET_NAME = "RAGEVALUATION-HJKMY/ragbench_emanual_400row"  
SPLITS_TO_SAMPLE = ['train']

original_dataset = load_dataset("rungalileo/ragbench", "emanual")

new_dataset_dict = {}
for split in SPLITS_TO_SAMPLE:
    if split in original_dataset:
        filtered_split = original_dataset[split].filter(lambda x: x['adherence_score'] == True)
        sampled_split = filtered_split.select(range(min(400, len(filtered_split))))  # Ensure there are enough rows
        new_dataset_dict[split] = sampled_split

new_dataset = DatasetDict(new_dataset_dict)


login(token=os.getenv('HF_TOKEN'))

new_dataset.push_to_hub(NEW_DATASET_NAME)

print(f"Dataset uploaded successfully to {NEW_DATASET_NAME}")

In [9]:
import pandas as pd
from datasets import Dataset
NEW_DATASET_NAME = "RAGEVALUATION-HJKMY/ragbench_delucionqa_400row"  
SPLITS_TO_SAMPLE = 'train'
# load local train.csv
# the first title line: sample_id,Retreival Setting,Question,Context,Answer,Answer_sent_tokenized,Sentence_labels,Label,Answerable,Does_not_answer
local_data = pd.read_csv("/Users/likai/IdeaProjects/RAG-LLM-Metric/personal/kai/train.csv")
# keep data that Label = Not Hallucinated and Answerable = TRUE
local_filtered = local_data[(local_data['Label'] == 'Not Hallucinated') & (local_data['Answerable'] == True)]
local_filtered = local_filtered.add_prefix("delu_")
# local data from ragbench
ragbench_dataset = load_dataset("rungalileo/ragbench", "delucionqa")
filtered_rag = ragbench_dataset[SPLITS_TO_SAMPLE].filter(lambda x: x['adherence_score'] == True)
rag_df = filtered_rag.to_pandas()
rag_df = rag_df[['question', 'documents','response','documents_sentences','response_sentences','sentence_support_information','unsupported_response_sentence_keys',
                 'overall_supported_explanation','relevance_explanation','all_relevant_sentence_keys',
                 'all_utilized_sentence_keys','relevance_score','utilization_score','completeness_score']]
# keep and deduplicate the question and documents column of filtered_rag
rag_df = rag_df.drop_duplicates(['question'])

# left join the local data with the filtered_rag where filtered_rag['question'] = local['question']
merged_data = local_filtered.merge(rag_df, left_on="delu_Question", right_on="question")
merged_data.rename(columns={'delu_sample_id': 'id'}, inplace=True)
# keep 400 rows data
final_dataset = Dataset.from_pandas(merged_data.iloc[:400])

print(final_dataset)

new_dataset_dict[SPLITS_TO_SAMPLE] = final_dataset

new_dataset = DatasetDict(new_dataset_dict)


login(token=os.getenv('HF_TOKEN'))

new_dataset.push_to_hub(NEW_DATASET_NAME)

print(f"Dataset uploaded successfully to {NEW_DATASET_NAME}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Dataset({
    features: ['id', 'delu_Retreival Setting', 'delu_Question', 'delu_Context', 'delu_Answer', 'delu_Answer_sent_tokenized', 'delu_Sentence_labels', 'delu_Label', 'delu_Answerable', 'delu_Does_not_answer', 'question', 'documents', 'response', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'overall_supported_explanation', 'relevance_explanation', 'all_relevant_sentence_keys', 'all_utilized_sentence_keys', 'relevance_score', 'utilization_score', 'completeness_score'],
    num_rows: 400
})


Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


Dataset uploaded successfully to RAGEVALUATION-HJKMY/ragbench_delucionqa_400row
